In [1]:
## Imports
import numpy as np
import tensorflow as tf
import time
import os
import re

from tensorflow import keras
from tensorflow.keras.models import Sequential#, Model
from tensorflow.keras.layers import Dense, Dropout#, Flatten, Input, Activation, Bidirectional
from tensorflow.keras.layers import TimeDistributed
#from tensorflow.keras.layers import Lambda, concatenate
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN#, RNN, Conv1D, MaxPooling1D

from tensorflow.keras.optimizers import Adam#, SGD, RMSprop, Nadam
from tensorflow.keras import backend as K
#from tensorflow.keras import regularizers

# from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import *

#import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
# import pandas_datareader as pdr
# import yfinance as yf
# import quandl

from keras.models import load_model
# from joblib import load
import json

### Game Rules
When it's your turn to play, you must pick a letter. This letter added to the previous letters cannot form a word or you lose. At the same time, there has to be a word that starts with the letters played.

In [15]:

import bisect

def get_lexicon(language):
    '''
    Defines the lexicon and alphabet to be used.
    Argument:
        String - language: Language of lexicon
    Returns
        lex - List containing the words in the specified lexicon
        alphabet - String containing all acceptable letters in alphabet
    '''
    if language=='SE':
        filename = 'swedish_lexicon.txt'
        alphabet = 'abcdefghijklmnopqrstuvwxyzåäö'
    elif language=='EN':
        filename = 'words_alpha.txt'
        alphabet = 'abcdefghijklmnopqrstuvwxyz'
    with open(filename, encoding="utf-8") as f:
        all_lines = f.readlines()
    lex = []
    for line in all_lines:
        lex.append(line.rstrip())
    
    return lex, alphabet

def is_word(letters, lex):
    return letters in lex

def has_prefix(prefix, lex):
    '''
    Checks if any word in lex starts with prefix. Call get_word to get an example of a word if this function returns True.
    Arguments:
        prefix: String containing characters you wish to call.

    Returns:
       True if any word in lex starts with prefix, else False
    '''
    i = bisect.bisect_left(lex, prefix)
    if i < len(lex) and lex[i].startswith(prefix):
        return True
    return False

def get_example(letters, lex):
    '''
    Gets an example of a word that starts with letters, or empty string if none.
    '''
    i = bisect.bisect_left(lex, letters)
    if i < len(lex) and lex[i].startswith(letters):
        return lex[i]
    return ''


### Bot definitions

In [9]:
import random

def bogo_bot(alphabet):
    '''
    Returns random letter in alphabet
    '''
    return random.choice(alphabet)

def is_vowel(char): ## Helper function for vowel_bot
    return char in 'aouåeiyäö'
def vowel_bot(letters, alphabet):
    '''
    Returns random vowel if previous letter was consonant and vice versa
    '''
    if len(letters) == 0:
        return bogo_bot(alphabet)
    if is_vowel(letters[-1]):
        return random.choice('bcdfghjklmnpqrstvwxz')
    else:
        return random.choice('aouåeiyäö')
    
def shallow_bot(letters, lex, alphabet):
    '''
    Checks if current letter combination is a word or if it's the start of any word,
    and picks a random letter that is the continuation of a word without finishing it.
    If no such letter exists, picks a random letter according to vowel_bot()
    Returns:
        character for the action
    '''
    if is_word(letters, lex): # If letters played already forms a word, call it
        return '0'
    if not has_prefix(letters, lex): # If letters played does not form a word, call it
        return '1'
    
    # Else, find all possible next letters
    allowed_chars = []
    for letter in alphabet:
        if has_prefix(letters+letter, lex) and not is_word(letters+letter, lex):
            allowed_chars.append(letter)
    
    if len(allowed_chars) > 0:
        return random.choice(allowed_chars)
    else:
        return vowel_bot(letters, alphabet)
    



### Play game

In [ ]:
n_players = 2
language = 'SE'

lex, alphabet = get_lexicon(language)

letters=''
curr_player = random.randint(0,n_players-1) # 0: user

while True:
    print('#############')
    print('Currently played:')
    print(letters)

    if curr_player == 0:
        print('You are up, type a letter to play, 0 to call a word, or 1 to call no word.')
        input_char = input()
    else: # You're not up, bot plays
        input_char = shallow_bot(letters, lex, alphabet)

    if input_char == '0':
        print(f'Player thinks {letters} is a word...')
        if is_word(letters, lex):
            print("...and it is.")
            user_victory = (curr_player == 0)
        else:
            print('... but it is not.')
            user_victory = (curr_player != 0)
        break
    elif input_char == '1':
        print(f'Player thinks {letters} is not a start of a word...')
        word = get_example(letters, lex)
        if word != '':
            print(f'...but it is, for example: {word}.')
            user_victory = (curr_player != 0)
        else:
            print('...and it is not.')
            user_victory = (curr_player == 0)
        break

    letters += input_char
    
    curr_player = (curr_player + 1) % n_players

print('Game over')
if user_victory:
    print('You won!')
else:
    print('You lost, better luck next time!')

#############
Currently played:

You are up, type a letter to play, 0 to call a word, or 1 to call no word.
#############
Currently played:
t
Player thinks t is a word...
...and it is.
Game over
You lost, better luck next time!
